# Task 2

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from sparkmeasure import StageMetrics
from pyspark.sql import functions as F
from operator import add
import operator
from functools import reduce
from datetime import datetime, timedelta

spark = SparkSession.builder.appName("task2").getOrCreate()
stagemetrics = StageMetrics(spark)

dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
year = 2000

In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 

In [0]:
def avg(ls): return reduce(lambda x, y: float(x) + float(y), ls) / len(ls)
  
def convert_24_hr(flight):
    flight[6]=str(flight[6])
    flight[8]=str(flight[8])
    if len(flight[6])<4:
      if len(flight[6])==1:flight[6]=="000"+flight[6]
      elif len(flight[6])==2:flight[6]=="00"+flight[6]
      else:flight[6]=="0"+flight[6]
    if len(flight[8])<4:
      if len(flight[8])==1:flight[8]=="000"+flight[8]
      elif len(flight[8])==2:flight[8]=="00"+flight[8]
      else:flight[8]=="0"+flight[8]
    if flight[6] == "2400": flight[6] = "0000"
    if flight[8] == "2400": flight[8] = "0000"
    return flight
  
def compute_delay(time_departed,time_scheduled):
    time_departed = datetime.strptime(str(time_departed), '%H%M')
    time_scheduled = datetime.strptime(str(time_scheduled), '%H%M')
    delay = time_departed - time_scheduled
    if delay.total_seconds()/(60*60) > 12: delay -= timedelta(hours=24)
    elif delay.total_seconds()/(60*60) < -12: delay += timedelta(hours=24)
    return delay.total_seconds()/60
    
def task_2_rdd(spark_session, flights_path, airlines_path, year):
    flights_rdd = spark.sparkContext.textFile(flights_path).map(lambda x: x.split(",")) 
    airlines_rdd = spark.sparkContext.textFile(airlines_path).map(lambda x: x.split(",")) 
    # airlines from USA
    airlines_rdd = airlines_rdd.filter(lambda x : x[2] == "United States")
    # Carrier code and name columns only
    airlines_rdd = airlines_rdd.map(lambda x: (x[0], x[1]))
    # Takes only rows with flight_date == year
    flights_rdd = flights_rdd.filter(lambda x: x[2].split("-")[0] == str(year))
    # Filter null actual_departure_time
    flights_rdd = flights_rdd.filter(lambda x: x[8]!="")
    flights_rdd = flights_rdd.filter(lambda x: x[6]!="")
    # Change 24:00:00 to 00:00:00 time
    flights_rdd = flights_rdd.map(convert_24_hr)
    # Take only carrier_code, scheduled_depature_time, actual_departure_time
    flights_rdd = flights_rdd.map(lambda x: (x[0], x[6], x[8]))

    # carrier_code, delay
    flights_rdd = flights_rdd.map(lambda x: (x[0], compute_delay(x[2], x[1])))
    # filter out delays greater than 0
    flights_rdd = flights_rdd.filter(lambda x: (x[1] > 0))
    
    flights_rdd = flights_rdd.groupByKey()
    flights_rdd_result = flights_rdd.map(lambda x: (x[0], (len(x[1]), round(avg(x[1]),2), min(x[1]), max(x[1]))))
    joined_rdd = flights_rdd_result.join(airlines_rdd)    
    final_joined_rdd = joined_rdd.map(lambda x: (x[1][1], x[1][0][0], x[1][0][1], x[1][0][2], x[1][0][3])).sortBy(lambda x: x[0])
                                                #name     #numDelay   #avgDelay   #min        #max
    results = final_joined_rdd.collect()
    results.sort(key=operator.itemgetter(0))                                                                                     
                                                                                                   
    for i in range(len(results)): 
      print(str(results[i][0])+'\t'+
            str(results[i][1])+'\t'+
            str(results[i][2])+'\t'+
            str(results[i][3])+'\t'+
            str(results[i][4]))

In [0]:
stagemetrics.begin()

task_2_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_small.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc.	66	25.18	1.0	119.0
American Airlines Inc.	252	30.97	1.0	379.0
Continental Air Lines Inc.	127	21.87	1.0	216.0
Delta Air Lines Inc.	389	23.88	1.0	336.0
Northwest Airlines Inc.	144	33.78	1.0	237.0
Southwest Airlines Co.	358	30.04	2.0	213.0
US Airways	237	28.46	1.0	496.0
United Airlines	376	33.13	1.0	397.0

In [0]:
stagemetrics.begin()

task_2_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_medium.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc.	595	32.06	1.0	324.0
American Airlines Inc.	2740	30.07	1.0	720.0
Continental Air Lines Inc.	1258	28.09	1.0	485.0
Delta Air Lines Inc.	3711	21.57	1.0	503.0
Northwest Airlines Inc.	1616	30.51	1.0	520.0
Southwest Airlines Co.	3635	27.67	1.0	350.0
US Airways	2683	28.48	1.0	429.0
United Airlines	3653	35.05	1.0	690.0

In [0]:
stagemetrics.begin()

task_2_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_large.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc.	6028	30.49	1.0	588.0
American Airlines Inc.	27231	30.16	1.0	720.0
Continental Air Lines Inc.	12880	29.31	1.0	665.0
Delta Air Lines Inc.	38169	22.31	1.0	606.0
Northwest Airlines Inc.	15867	29.78	1.0	706.0
Southwest Airlines Co.	36874	28.02	1.0	465.0
US Airways	26197	27.95	1.0	618.0
United Airlines	36834	34.28	1.0	718.0

In [0]:
stagemetrics.begin()

task_2_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_massive.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

stagemetrics.end()

Alaska Airlines Inc.	47835	30.48	1.0	696.0
American Airlines Inc.	218659	30.18	1.0	720.0
Continental Air Lines Inc.	101973	28.48	1.0	720.0
Delta Air Lines Inc.	303248	21.87	1.0	700.0
Northwest Airlines Inc.	126611	29.64	1.0	717.0
Southwest Airlines Co.	293942	28.12	1.0	635.0
US Airways	208389	28.07	1.0	714.0
United Airlines	295520	34.35	1.0	720.0